# Example 2: Simple 2D cell signaling model

We model a reaction between the cell interior and cell membrane in a 2D geometry:
- Cyto - 2D cell "volume"
- PM - 1D cell boundary (represents plasma membrane)

Model from [Rangamani et al, 2013, Cell](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3874130/). A cytosolic species, "A", reacts with a species on the PM, "B", to form a new species on the PM, "X".


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
img_A = mpimg.imread('axb-diagram.png')
plt.imshow(img_A)
plt.axis('off')

Imports and logger initialization:

In [ ]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib
import logging
import gmsh # must be imported before pyvista is dolfin is imported first

from smart import config, common, mesh, model, mesh_tools, visualization
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

First, we define the various units for use in the model.

In [ ]:
# Aliases - base units
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
surf_unit = molecule / um**2
flux_unit = molecule / (um * sec)
edge_unit = molecule / um

Next we generate the model by assembling the compartment, species, parameter, and reaction containers (see Example 1 for or API documentation for more details).

In [ ]:
# =============================================================================================
# Compartments
# =============================================================================================
# name, topological dimensionality, length scale units, marker value
Cyto = Compartment("Cyto", 2, um, 1)
PM = Compartment("PM", 1, um, 3)
cc = CompartmentContainer()
cc.add([Cyto, PM])

# =============================================================================================
# Species
# =============================================================================================
# name, initial concentration, concentration units, diffusion, diffusion units, compartment
A = Species("A", 1.0, surf_unit, 100.0, D_unit, "Cyto")
X = Species("X", 1.0, edge_unit, 1.0, D_unit, "PM")
B = Species("B", 0.0, edge_unit, 1.0, D_unit, "PM")
sc = SpeciesContainer()
sc.add([A, X, B])

# =============================================================================================
# Parameters and Reactions
# =============================================================================================

# Reaction of A and X to make B (Cyto-PM reaction)
kon = Parameter("kon", 1.0, 1/(surf_unit*sec))
koff = Parameter("koff", 1.0, 1/sec)
r1 = Reaction("r1", ["A", "X"], ["B"], 
              param_map={"on": "kon", "off": "koff"}, 
              species_map={"A": "A", "X": "X", "B": "B"})

pc =ParameterContainer()
pc.add([kon, koff])
rc = ReactionContainer()
rc.add([r1])

Now we create an circular mesh (mesh built using gmsh in `smart.mesh_tools`), along with marker functions `mf2` and `mf1`.

In [ ]:
# Create mesh
h_ellipse = 0.1
major_rad = 1.0
minor_rad = 1.0
inner_tag = 1
edge_tag = 3
ellipse_mesh, mf1, mf2 = mesh_tools.DemoEllipseMesh(major_rad, minor_rad, h_ellipse, inner_tag, edge_tag)
# fig = plt.figure()
# img_mesh = mpimg.imread('example2-mesh.png')
# plt.imshow(img_mesh)
# plt.axis('off')
# fig.set_size_inches(8, 8)
visualization.plot_dolfin_mesh(ellipse_mesh, mf2, view_xy=True)

Write mesh and meshfunctions to file, then create `mesh.ParentMesh` object.

In [ ]:

mesh_folder = pathlib.Path("ellipse_mesh")
mesh_folder.mkdir(exist_ok=True)
mesh_file = mesh_folder / "ellipse_mesh.h5"
hdf5 = d.HDF5File(ellipse_mesh.mpi_comm(), str(mesh_file.with_suffix(".h5")), "w")
hdf5.write(ellipse_mesh, "/mesh")
hdf5.write(mf2, "/mf2")
hdf5.write(mf1, "/mf1")
# For visualization of domains
d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf2").with_suffix(".pvd"))) << mf2
d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf1").with_suffix(".pvd"))) << mf1

parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_file),
    mesh_filetype="hdf5",
    name="parent_mesh",
)

Initialize model and solvers.

In [ ]:
configCur = config.Config()
configCur.solver.update(
    {
        "final_t": 5.0,
        "initial_dt": 0.05,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)

modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
modelCur.initialize(initialize_solver=False)
modelCur.initialize_discrete_variational_problem_and_solver()

Save model information to .pkl file and write initial conditions to file.

In [ ]:
modelCur.to_pickle('modelCur.pkl')
results = dict()
result_folder = pathlib.Path("resultsEllipse")
result_folder.mkdir(exist_ok=True)
for species_name, species in modelCur.sc.items:
    results[species_name] = d.XDMFFile(
        modelCur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)

Solve the system until `modelCur.t > modelCur.final_t`.

In [ ]:
tvec = [0]
avg_A = [A.initial_condition]
while True:
    # Solve the system
    modelCur.monolithic_solve()
    # Save results for post processing
    for species_name, species in modelCur.sc.items:
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
    dx = d.Measure("dx",domain = modelCur.cc['Cyto'].dolfin_mesh)
    int_val = d.assemble(modelCur.sc['A'].u['u']*dx)
    volume = d.assemble(1.0*dx)
    avg_A.append(int_val / volume)
    tvec.append(modelCur.t)
    # End if we've passed the final time
    if modelCur.t >= modelCur.final_t:
        break

Now we plot the concentration of A in the cell over time and compare this to analytical predictions for high value of the diffusion coefficient.

In [ ]:
plt.plot(tvec, avg_A, label='SMART simulation')
plt.xlabel('Time (s)')
plt.ylabel('A concentration (uM)')
SA_vol = 4/(major_rad + minor_rad)
root_vals = np.roots([-kon.value,
                      -kon.value*X.initial_condition*SA_vol - koff.value + kon.value*A.initial_condition,
                      koff.value*A.initial_condition])
ss_pred = root_vals[root_vals > 0]
plt.plot(tvec, np.ones(len(avg_A))*ss_pred, '--', label='Steady-state analytical prediction')
plt.legend()
assert np.abs(ss_pred-avg_A[-1])/ss_pred < 0.1/100

Plot A concentration in the cell at the final time point.

In [ ]:
visualization.plot(modelCur.sc["A"].u["u"], view_xy=True)

Now iterate over three different cell aspect ratios and compare dynamics between cases.

In [ ]:
aspect_ratios = [1, 1.5, 2, 4]
ss_calc = [avg_A[-1]]
for i in range(1, len(aspect_ratios)):
    # Create mesh
    h_ellipse = 0.1
    major_rad = 1.0*aspect_ratios[i]
    minor_rad = 1.0/aspect_ratios[i]
    inner_tag = 1
    edge_tag = 3
    ellipse_mesh, mf1, mf2 = mesh_tools.DemoEllipseMesh(major_rad, minor_rad, h_ellipse, inner_tag, edge_tag)
    mesh_folder = pathlib.Path(f"ellipse_mesh_AR{aspect_ratios[i]}")
    mesh_folder.mkdir(exist_ok=True)
    mesh_file = mesh_folder / "ellipse_mesh.h5"
    hdf5 = d.HDF5File(ellipse_mesh.mpi_comm(), str(mesh_file.with_suffix(".h5")), "w")
    hdf5.write(ellipse_mesh, "/mesh")
    hdf5.write(mf2, "/mf2")
    hdf5.write(mf1, "/mf1")
    # For visualization of domains
    d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf2").with_suffix(".pvd"))) << mf2
    d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf1").with_suffix(".pvd"))) << mf1
    parent_mesh = mesh.ParentMesh(
        mesh_filename=str(mesh_file),
        mesh_filetype="hdf5",
        name="parent_mesh",
    )
    configCur = config.Config()
    configCur.solver.update(
        {
            "final_t": 5.0,
            "initial_dt": 0.05,
            "time_precision": 6,
            "use_snes": True,
            "print_assembly": False,
        }
    )
    modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
    modelCur.initialize(initialize_solver=False)
    modelCur.initialize_discrete_variational_problem_and_solver()
    results = dict()
    result_folder = pathlib.Path("resultsEllipse_AR{aspect_ratios[i]}")
    result_folder.mkdir(exist_ok=True)
    for species_name, species in modelCur.sc.items:
        results[species_name] = d.XDMFFile(
            modelCur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
        )
        results[species_name].parameters["flush_output"] = True
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
        tvec = [0]
    avg_A = [A.initial_condition]
    while True:
        # Solve the system
        modelCur.monolithic_solve()
        # Save results for post processing
        for species_name, species in modelCur.sc.items:
            results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
        dx = d.Measure("dx",domain = modelCur.cc['Cyto'].dolfin_mesh)
        int_val = d.assemble(modelCur.sc['A'].u['u']*dx)
        volume = d.assemble(1.0*dx)
        avg_A.append(int_val / volume)
        tvec.append(modelCur.t)
        # End if we've passed the final time
        if modelCur.t >= modelCur.final_t:
            break
    plt.plot(tvec, avg_A)
    ss_calc.append(avg_A[-1])

Compare steady-state values with those from past runs using SMART (regression test):

In [ ]:
ss_stored = [0.4139461050700552, 0.38031028434564446, 0.326981289191557, 0.18809307969607147]
assert max((np.array(ss_stored) - np.array(ss_calc))/np.array(ss_stored)) < .01/100